In [9]:
from google.colab import files
upload = files.upload()

Saving utils.py to utils.py


In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
import cv2
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
import os 
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import tensorflow as tf

In [4]:
path = './'

In [5]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [6]:
!kaggle datasets download -d sudharshann/pcb-defect-dataset

100% 1.10G/1.11G [00:37<00:00, 55.9MB/s]
100% 1.11G/1.11G [00:37<00:00, 31.6MB/s]


In [7]:
!unzip pcb-defect-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_05_1_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_05_2_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_05_3_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_06_1_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_06_2_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_07_1_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_07_2_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_07_3_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_07_4_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_07_5_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_08_1_600.jpg  
  inflating: VOC_PCB/JPEGImages/rotation_270_light_04_mouse_bite_08_2

In [8]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
image_path = os.path.join(path, 'VOC_PCB/JPEGImages')
print(image_path)
annotate_path = os.path.join(path, 'VOC_PCB/Annotations')
print(annotate_path)

./VOC_PCB/JPEGImages
./VOC_PCB/Annotations


In [10]:
from utils import *

  0%|          | 0/10668 [00:00<?, ?it/s]

  0%|          | 0/10668 [00:00<?, ?it/s]

In [11]:
bbox_for_all_image = bbox_for_all_image[:5334]
boxarr=boxarr[:5334]

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((bbox_for_all_image,boxarr))

In [13]:
n=1500
dataset = dataset.shuffle(n)
train_dataset = dataset.take(int(n*0.9)).batch(100)
test_dataset = dataset.skip(int(n*0.9)).batch(100)

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [15]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):

    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)

    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)

    outputs = Conv2D(7, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

input_img = Input((128, 128, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
model.compile(optimizer=Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  160         ['img[0][0]']                    
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 128, 128, 16  64         ['conv2d_1[0][0]']               
 rmalization)                   )                                                             

In [ ]:
callback = [EarlyStopping(patience=10, verbose=1),ReduceLROnPlateau(factor=0.001, patience=10, min_lr=0.0001, verbose=1),
             ModelCheckpoint('model best.h5', verbose=1, save_best_only=True, save_weights_only=False,
                             monitor='val_loss',save_freq='epoch',mode='loss')]

history = model.fit(train_dataset,epochs=2000,validation_data=test_dataset,callbacks=[callback],batch_size=32,)

Epoch 1/2000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


14/14 [==============================] - ETA: 0s - loss: 1.7592 - accuracy: 0.3331 
Epoch 1: val_loss improved from inf to 4.30727, saving model to model best.h5
14/14 [==============================] - 297s 22s/step - loss: 1.7592 - accuracy: 0.3331 - val_loss: 4.3073 - val_accuracy: 0.3693 - lr: 0.0010
Epoch 2/2000
14/14 [==============================] - ETA: 0s - loss: 1.3477 - accuracy: 0.7396 
Epoch 2: val_loss did not improve from 4.30727
14/14 [==============================] - 294s 22s/step - loss: 1.3477 - accuracy: 0.7396 - val_loss: 23.4159 - val_accuracy: 0.3604 - lr: 0.0010
Epoch 3/2000
14/14 [==============================] - ETA: 0s - loss: 1.0270 - accuracy: 0.9608 
Epoch 3: val_loss did not improve from 4.30727
14/14 [==============================] - 292s 21s/step - loss: 1.0270 - accuracy: 0.9608 - val_loss: 9.8916 - val_accuracy: 0.6413 - lr: 0.0010
Epoch 4/2000
14/14 [==============================] - ETA: 0s - loss: 0.8023 - accuracy: 0.9912 
Epoch 4: val_loss im

In [ ]:
model.save('pcbdefect.h5')